<a id='TOC'></a>

# Cleaning Data (Part 1)
The purpose of this notebook is to read in raw excel data for multiple years, append them into a single dataframe.

# Table of Contents
1. [Setting up Python](#SettingUp)
    
    a. [Setting the Location](#SettingLoc)
    
    b. [Importing Necessary Packages](#ImportingPackages)
    
    c. [Preparing for a Save](#PreparingSave)
    

2. [Handling Columns](#HandlingColumns)
    
    a. [Find Unique Column Names](#FindUniqueCol)
    
    b. [Eliminate Unnecessary Columns](#DropCol)
    
    c. [Combine Synonyms](#CombineCol)

3. [Reading and Appending Data](#ReadingAppendingData)

4. [Exporting Data](#ExportingData)

<a id='SettingUp'></a>

# Setting up Python
[Top](#TOC)

[Setting the Location](#SettingLoc)
    
[Importing Necessary Packages](#ImportingPackages)
    
[Getting Data](#GettingData)
    
[Preparing for a Save](#PreparingSave)

<a id='ImportingPackages'></a>

## Importing Necessary Packages

[Top](#TOC)

[Setting Up Python](#SettingUp)

Here we import necessary packages. 
This chunk may take a while.

In [1]:
import pandas as pd
import numpy as np
import glob,os
from liz_number import lizsort,mindate,smallest,validate
from liz_toes import make_str,label_pattern, replace_pattern,report_pattern

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_config_file(world_readable=True)

# increase print limit
pd.options.display.max_rows = 99999

<a id='functions'></a>

## Functions
[Top](#TOC)

[Setting Up Python](#SettingUp)

In [2]:
def xlcolshape(file):
    """xlcolshape takes a file name as a string and returns the shape of the excel file"""
    return pd.read_excel(file).shape

In [3]:
def xlsheets(file):
    """The function takes a string, *file*, and returns the number and names of sheets in that file.\
    The function relies on the pandas package."""
    tmp = pd.ExcelFile(file)
    res = {'file_name':file,'num_sheets':len(tmp.sheet_names),'names': tmp.sheet_names}
    return res

In [4]:
def xluniquecol2(file, header = 0):
    tmp = []
    for sheet in pd.ExcelFile(file).sheet_names:
        if (('species' in pd.read_excel(file,sheet_name=sheet, header = header).columns)\
            or('Species' in pd.read_excel(file,sheet_name=sheet, header = header).columns)):
            try:
                tmp = list(set(tmp+list(pd.read_excel(file,sheet_name=sheet).columns)))
                print("Doing stuff you asked me to do for file \'{}\',sheet \'{}\' programmer person."\
                      .format(file,sheet))
                res = tmp
            except:
                print("This didn't work for file {}, sheet {}".format(file,sheet))
        else:
            print("Check columns for file {}.".format(file))
            res = None
    return res
            

In [5]:
def colmatchtodict(x,series, dictsource, key= None):
    """This takes a string, x, and a looks for values in a series that match that contain that string.
    Those values which match are returned as values in a python dict for the key, key."""
    
    assert isinstance(series,pd.Series)
    if key is None:
        key = x
    tmp = series[series.astype(str).str.contains(x,case = False)].tolist()
    dictsource[key] = tmp
    return dictsource
    

In [6]:
def findsyn (name,dictionary, verbose = True):
    """
    *findsyn* checks searches the values of the dict *dictionary* for the string, *name* and returns 
    the key for the key,value pair to which *name* belongs.
    """
    tmp = pd.DataFrame({'preferredcol':list(dictionary.keys()),'synonymns':list(dictionary.values())})
    try:
        res = list(tmp.preferredcol[tmp.synonymns.apply(lambda x:name in x)])[0]
    except:
        res = None
        if verbose == True:
            print("No value matching \"{}\" was found in the dictionary.".format(name))
    return res

In [7]:
def readnclean(x,dictionary,dtype=None):
    """
    This function reads an excel file, renames columns deemed synonymous according to a dict,
    *dictionary*, and drops unnecessary columns before returning the cleaner dataframe.
    """
    tmp = pd.read_excel(x,dtype=dtype)
    tmp.columns = pd.Series(tmp.columns).map(lambda x:dictionary[x])
    dropidx =[None==col for col in list(tmp.columns)]
    tmp=tmp.drop(columns=tmp.columns[dropidx])
    
    return tmp

In [8]:
def namefile(name, tzadjust=5,tzdirection = '-', adjprecision='minutes', filetype = 'csv'):
    """takes a filename and filetype, and adds a timestamp adjusted relative to gmt to a precision 
    and returns a string that concatenates them."""
    assert isinstance(name,str),"'name' must be of type str."
    assert isinstance(tzadjust,int),"'tzadjust' must be of type int"
    assert adjprecision in ['date','hour','minutes','seconds', 'max'], "'adjprecision' must be either \
    'date', hour','minutes','seconds', or 'max'"
    precision= {'max':None,'seconds':-7,'minutes':-9, 'hours':-14,'date':-20}
    if tzdirection== '-':
        timestamp = (pd.to_datetime('now')-pd.Timedelta(hours=tzadjust))
    else:
        timestamp = (pd.to_datetime('now')+pd.Timedelta(hours=int(tzadjust[1:])))
    timestamp = str(timestamp).replace(':','hrs',1).replace(':','min',1)
    timestamp = timestamp[:precision[adjprecision]]
    filename = name+'_' + timestamp+ '.' +filetype
    return filename


<a id='PreparingSave'></a>

## Preparing for a Save
[Top](#TOC)

[Setting up Python](#SettingUp)

<a id='SettingLoc'></a>

## Setting the Location
[Top](#TOC)

[Setting Up Python](#SettingUp)

These chunks identify the locations from which we can get data and to which we can save data.

### Source Data
Raw data can be found in the following locations:

In [9]:
# sourceDataPers = 'C:/Users/Christopher/Google Drive/TailDemography/outputFiles'
sourceDataBig = 'S:/Chris/TailDemography/TailDemography/Raw Data'
# sourceBlack = 'C:/Users/test/Desktop'

Now we change the working directory to the source path.

In [10]:
os.chdir(sourceDataBig)

### Output Data
The cleaned data will be saved to one of these locations:

In [11]:
# outputPers = 'C:/Users/Christopher/Google Drive/TailDemography/outputFiles'
outputBig = 'S:/Chris/TailDemography/TailDemography/Cleaned Combined Data'
# outputBlack = 'C:/Users/test/Desktop'

<a id='HandlingColumns'></a>

# Handling Columns
[Top](#TOC)

We don't have to look in the multiple-sheet file.  It's clear that we'll have to identify a common set of columns prior to combining these files.  Let's define a few functions to help us do this.

We will want to do the following:
1. [Find Unique Column Names](#FindUniqueCol)
2. [Eliminate Unnecessary Columns](#DropCol)
3. [Combine Synonyms](#CombineCol)

Here we use search the source path to locate and eventually read the raw data into our notebook.

In [12]:
rawfiles = glob.glob('*.xls*')
rawfiles

['CC 2000-03-modified from CC-SJ 00-03 final.xls',
 'CC 2004.xlsx',
 'CC 2015 - captures.xls',
 'CC 2016 - captures.xls',
 'CC 2017 Lizards - 3viii17 captures and obs.xls',
 'xCC2005x.xls',
 'xCC2006x.xls',
 'xCC2007x.xls',
 'xCC2008x.xls',
 'xCC2009x.xls',
 'xCC2010x.xlsx',
 'xCC2011x.xls',
 'xCC2012x.xls',
 'xCC2013x.xls',
 'xCC2014x.xlsx']

Let's inspect these data.  We will begin by looking at the number of columns and rows. In each file. To do this first we use the function, *xlsheets*, defined in the [Functions](#functions) section.

You can see that when we call *xlsheets* on the first file in our list of files, it returns the total number of sheets and names of each sheet.

In [13]:
xlsheets(rawfiles[0])

{'file_name': 'CC 2000-03-modified from CC-SJ 00-03 final.xls',
 'num_sheets': 4,
 'names': ['2000', '2001', '2002', '2003']}

We will apply this to all of the files in raw files to inspect the data at the highest level.

In [14]:
filelayout = pd.Series(rawfiles).apply(xlsheets)
filelayout

0     {'file_name': 'CC 2000-03-modified from CC-SJ ...
1     {'file_name': 'CC 2004.xlsx', 'num_sheets': 1,...
2     {'file_name': 'CC 2015 - captures.xls', 'num_s...
3     {'file_name': 'CC 2016 - captures.xls', 'num_s...
4     {'file_name': 'CC 2017 Lizards - 3viii17 captu...
5     {'file_name': 'xCC2005x.xls', 'num_sheets': 1,...
6     {'file_name': 'xCC2006x.xls', 'num_sheets': 1,...
7     {'file_name': 'xCC2007x.xls', 'num_sheets': 2,...
8     {'file_name': 'xCC2008x.xls', 'num_sheets': 1,...
9     {'file_name': 'xCC2009x.xls', 'num_sheets': 1,...
10    {'file_name': 'xCC2010x.xlsx', 'num_sheets': 1...
11    {'file_name': 'xCC2011x.xls', 'num_sheets': 1,...
12    {'file_name': 'xCC2012x.xls', 'num_sheets': 1,...
13    {'file_name': 'xCC2013x.xls', 'num_sheets': 1,...
14    {'file_name': 'xCC2014x.xlsx', 'num_sheets': 1...
dtype: object

In [15]:
filelayout = pd.DataFrame([list(file.values()) for file in filelayout])
filelayout.columns = ['file_name','num_sheets','sheetnames']
filelayout

,file_name,num_sheets,sheetnames
0,CC 2000-03-modified from CC-SJ 00-03 final.xls,4,"[2000, 2001, 2002, 2003]"
1,CC 2004.xlsx,1,[2004 ]
2,CC 2015 - captures.xls,1,[2015]
3,CC 2016 - captures.xls,1,[2016]
4,CC 2017 Lizards - 3viii17 captures and obs.xls,1,[2017]
5,xCC2005x.xls,1,[2005]
6,xCC2006x.xls,1,[2006]
7,xCC2007x.xls,2,"[Sheet1, 2007]"
8,xCC2008x.xls,1,[2008]
9,xCC2009x.xls,1,[2009]


We will need to treat the 1st and 8th files in the source data differently when we read in the data.  Let's group the file names accordingly.

In [16]:
rawfiles_ms = [rawfiles[0],rawfiles[7]]
rawfiles_ss = list(set(rawfiles)- set(rawfiles_ms))

The names of files with multiple sheets are now in the variable *rawfiles_ms*.

In [17]:
rawfiles_ms

['CC 2000-03-modified from CC-SJ 00-03 final.xls', 'xCC2007x.xls']

The names of files with a single sheet are now in the variable *rawfiles_ss*.

In [18]:
rawfiles_ss

['xCC2012x.xls',
 'CC 2017 Lizards - 3viii17 captures and obs.xls',
 'CC 2016 - captures.xls',
 'xCC2005x.xls',
 'xCC2006x.xls',
 'xCC2008x.xls',
 'xCC2014x.xlsx',
 'xCC2011x.xls',
 'xCC2009x.xls',
 'xCC2010x.xlsx',
 'xCC2013x.xls',
 'CC 2004.xlsx',
 'CC 2015 - captures.xls']

Now let's take a look at the number of columns in each file. We'll start with the single sheet files, since this is the easiest.  We will use the function, *xlcolshape* to make this easier. 
When we call this function on the first of the single-sheet files, we can see that it returns a tuple in the format ('number of rows', 'number of columns'). The code for *xlcolshape* can be found in [Functions](#functions).

In [19]:
xlcolshape(rawfiles_ss[0])

(85, 19)

We will apply this funtion to the list of single-sheet files for our inspection.

In [20]:
pd.Series(rawfiles_ss).apply(xlcolshape)

0      (85, 19)
1     (798, 21)
2     (103, 21)
3     (202, 16)
4     (163, 16)
5     (134, 20)
6      (97, 19)
7      (64, 19)
8     (162, 16)
9      (99, 41)
10    (106, 19)
11    (479, 16)
12    (241, 19)
dtype: object

<a id='FindUniqueCol'></a>

## Finding Unique Columns
[Top](#TOC)

[Handling Columns](#HandlingColumns)

We'll use the function, *xluniqucol2* to extract column names and convert them to an approved set.  We'll use that function to allow us to only add unique names to a list of column names. 

Here is an example of how xluniquecol2 works for a file with one sheet.  You can find the code for *xluniquecol2* in [Functions](#functions).

In [21]:
xluniquecol2(rawfiles_ss[0])

Doing stuff you asked me to do for file 'xCC2012x.xls',sheet 'data' programmer person.


['Species',
 'Meters',
 'Toes',
 'Vial',
 'Time',
 'New/Recap',
 'TL',
 'RTL',
 'Mass',
 'Unnamed: 16',
 'Misc.',
 'SVL',
 'Sex',
 'Mark',
 'Painted',
 'Location',
 'Date',
 'Spotted',
 'Paint Mark']

Here is an example of how xluniquecol2 works for a file with multiple sheets.

In [22]:
xluniquecol2(rawfiles_ms[0])

Check columns for file CC 2000-03-modified from CC-SJ 00-03 final.xls.
Doing stuff you asked me to do for file 'CC 2000-03-modified from CC-SJ 00-03 final.xls',sheet '2001' programmer person.
Doing stuff you asked me to do for file 'CC 2000-03-modified from CC-SJ 00-03 final.xls',sheet '2002' programmer person.
Doing stuff you asked me to do for file 'CC 2000-03-modified from CC-SJ 00-03 final.xls',sheet '2003' programmer person.


['species',
 'mass',
 'location',
 'NEW/recap',
 'VIAL',
 'TL',
 'RTL',
 'Unnamed: 16',
 'misc',
 'paint mark',
 'meters',
 'SVL',
 'TOES',
 'sex',
 'TIME',
 'painted or not',
 'date']

Now we will create an empty set, *uniquecols2*, that will eventually contain the unique column names in all of the files.

We will append the unique column names from each file to *uniquecols2*.

In [23]:
tmp = pd.Series(rawfiles_ss).apply(xluniquecol2)
uniquecols2 = list()
for u in tmp:
    uniquecols2 = uniquecols2+u
uniquecols2 = list(set(uniquecols2))
uniquecols2

Doing stuff you asked me to do for file 'xCC2012x.xls',sheet 'data' programmer person.
Doing stuff you asked me to do for file 'CC 2017 Lizards - 3viii17 captures and obs.xls',sheet '2017' programmer person.
Doing stuff you asked me to do for file 'CC 2016 - captures.xls',sheet '2016' programmer person.
Doing stuff you asked me to do for file 'xCC2005x.xls',sheet '2005' programmer person.
Doing stuff you asked me to do for file 'xCC2006x.xls',sheet '2006' programmer person.
Doing stuff you asked me to do for file 'xCC2008x.xls',sheet '2008' programmer person.
Doing stuff you asked me to do for file 'xCC2014x.xlsx',sheet '2014' programmer person.
Doing stuff you asked me to do for file 'xCC2011x.xls',sheet 'Sheet1' programmer person.
Doing stuff you asked me to do for file 'xCC2009x.xls',sheet '2009' programmer person.
Doing stuff you asked me to do for file 'xCC2010x.xlsx',sheet 'Sheet1' programmer person.
Doing stuff you asked me to do for file 'xCC2013x.xls',sheet 'CC 2013 data' prog

[1,
 ' painted or not',
 'Vial',
 'location',
 'Toe 16',
 'Toe 14',
 'Toe 1',
 'VIAL',
 'Mass',
 'RTL',
 'Marked',
 'Toe 6',
 'Toe 18',
 'Toe 19',
 'SVL (mm)',
 'Location',
 'Spotted',
 'Species',
 'Meters',
 'Toe 5',
 'painted',
 'Toe 7',
 'Tail condition (1=intact; 2=autotomized; 3=regrown)',
 'Unnamed: 16',
 '2015 or earlier',
 'Toe 20',
 'SVL',
 'Toe 2',
 'sex',
 'Collectors',
 'Mark',
 'Toe 12',
 'Year',
 'painted or not',
 '1st Capture (year)',
 'Toe 11',
 'species',
 'mass (g)',
 'Years Alive (known)',
 'Toes',
 'Unnamed: 17',
 'NEW/recap',
 'misc/notes',
 'New/Recap',
 'TL',
 'Toe 8',
 'paint mark',
 'meters',
 'TOES',
 'TIME',
 'Toe 3',
 'Sex',
 'TL (mm)',
 'Painted',
 'Toe 9',
 'Toe 13',
 'mass',
 'Time',
 'RTL (mm)',
 'Toe 10',
 'misc',
 'Misc.',
 'Toe 15',
 'Toe 4',
 'Unnamed: 19',
 'Date',
 'date',
 'Toe 17',
 'Paint Mark']

<a id='DropCol'></a>

## Eliminate Unnecessary Columns
[Top](#TOC)

[Cleaning Data](#CleaningData)

[Handling Columns](#HandlingColumns)

Now we will try to identify unnecessary columns and eliminate them. Much of this will be done manually.

In [24]:
keepCol = ['species', 'date', 'sex', 'svl', 'tl', 'rtl', 'mass',
       'paint.mark', 'location', 'meters', 'new.recap', 'painted', 'misc',
       'vial', 'autotomized', 'sighting', 'toes']

In [25]:
set(pd.Series(keepCol).str.lower())-set(pd.Series(uniquecols2).str.lower())

{'autotomized', 'new.recap', 'paint.mark', 'sighting'}

In [26]:
set(pd.Series(uniquecols2).str.lower())-set(pd.Series(keepCol).str.lower())

{' painted or not',
 '1st capture (year)',
 '2015 or earlier',
 'collectors',
 'mark',
 'marked',
 'mass (g)',
 'misc.',
 'misc/notes',
 nan,
 'new/recap',
 'paint mark',
 'painted or not',
 'rtl (mm)',
 'spotted',
 'svl (mm)',
 'tail condition (1=intact; 2=autotomized; 3=regrown)',
 'time',
 'tl (mm)',
 'toe 1',
 'toe 10',
 'toe 11',
 'toe 12',
 'toe 13',
 'toe 14',
 'toe 15',
 'toe 16',
 'toe 17',
 'toe 18',
 'toe 19',
 'toe 2',
 'toe 20',
 'toe 3',
 'toe 4',
 'toe 5',
 'toe 6',
 'toe 7',
 'toe 8',
 'toe 9',
 'unnamed: 16',
 'unnamed: 17',
 'unnamed: 19',
 'year',
 'years alive (known)'}

Since data for years 2000-2003 are contained in the same Excel file we will have to treat this file differently than the others.

<a id='CombineCol'></a>

## Combining Synonymous Columns
[Top](#TOC)

[Cleaning Data](#CleaningData)

[Handling Columns](#HandlingColumns)

Once we have identified the columns we need to keep, we'll need to apply this list to the files as they are read into python by doing the following:

We will use a function, *colmatchtodict*,  to identify potential synonyms. Here's an example of how *colmatchtodict* works.  The code for this function can be found in [Functions](#functions).

In [27]:
coldict = {}

In [28]:
colmatchtodict('toes',pd.Series(uniquecols2),coldict, key = 'toes')

{'toes': ['Toes', 'TOES']}

Now let's see what happened when we apply this funtion to our, keepCol.

In [29]:
coldict = {}

In [30]:
pd.Series(keepCol).apply(lambda x: colmatchtodict(x=x,series=pd.Series(uniquecols2),dictsource=coldict))
coldict

{'species': ['Species', 'species'],
 'date': ['Date', 'date'],
 'sex': ['sex', 'Sex'],
 'svl': ['SVL (mm)', 'SVL'],
 'tl': ['RTL', 'TL', 'TL (mm)', 'RTL (mm)'],
 'rtl': ['RTL', 'RTL (mm)'],
 'mass': ['Mass', 'mass (g)', 'mass'],
 'paint.mark': ['paint mark', 'Paint Mark'],
 'location': ['location', 'Location'],
 'meters': ['Meters', 'meters'],
 'new.recap': ['NEW/recap', 'New/Recap'],
 'painted': [' painted or not', 'painted', 'painted or not', 'Painted'],
 'misc': ['misc/notes', 'misc', 'Misc.'],
 'vial': ['Vial', 'VIAL'],
 'autotomized': ['Tail condition (1=intact; 2=autotomized; 3=regrown)'],
 'sighting': [],
 'toes': ['Toes', 'TOES']}

We will manually adjust the values for 'tl'.

In [31]:
coldict['tl']=['TL (mm)', 'TL', 'tl']

Now we need to use this dict to relabel the columns we wish to keep.

We will use the function, *findsyn* to identify potential synonymous to the columnlabels in *keepcols* among the column labels in *uniquecols2*. 

Here is are a few examples of how *findsyn* works.  The code can be found in [Functions](#functions).

In [32]:
findsyn('RTi',coldict,verbose=False)

In [33]:
findsyn('RTi',coldict,verbose=True)

No value matching "RTi" was found in the dictionary.


In [34]:
findsyn('RTL',coldict,verbose=True)

'rtl'

Now we apply *findsyn* to *uniquecol* and create a column of synonyms.

In [35]:
uniquecols2df = pd.DataFrame({'uniquecols2':uniquecols2})
uniquecols2df['preferredcol'] = uniquecols2df.uniquecols2.apply(lambda x: findsyn(x,coldict,False))
uniquecols2df

,uniquecols2,preferredcol
0,1,None
1,painted or not,painted
2,Vial,vial
3,location,location
4,Toe 16,None
5,Toe 14,None
6,Toe 1,None
7,VIAL,vial
8,Mass,mass
9,RTL,rtl


Now we will turn this dataframe back into a dict so that we can easily use it to rename columns

In [36]:
uniquecols2df.index = uniquecols2df.uniquecols2
uniquecols2dict = pd.Series(uniquecols2df.preferredcol).to_dict()
uniquecols2dict

{1: None,
 ' painted or not': 'painted',
 'Vial': 'vial',
 'location': 'location',
 'Toe 16': None,
 'Toe 14': None,
 'Toe 1': None,
 'VIAL': 'vial',
 'Mass': 'mass',
 'RTL': 'rtl',
 'Marked': None,
 'Toe 6': None,
 'Toe 18': None,
 'Toe 19': None,
 'SVL (mm)': 'svl',
 'Location': 'location',
 'Spotted': None,
 'Species': 'species',
 'Meters': 'meters',
 'Toe 5': None,
 'painted': 'painted',
 'Toe 7': None,
 'Tail condition (1=intact; 2=autotomized; 3=regrown)': 'autotomized',
 'Unnamed: 16': None,
 '2015 or earlier': None,
 'Toe 20': None,
 'SVL': 'svl',
 'Toe 2': None,
 'sex': 'sex',
 'Collectors': None,
 'Mark': None,
 'Toe 12': None,
 'Year': None,
 'painted or not': 'painted',
 '1st Capture (year)': None,
 'Toe 11': None,
 'species': 'species',
 'mass (g)': 'mass',
 'Years Alive (known)': None,
 'Toes': 'toes',
 'Unnamed: 17': None,
 'NEW/recap': 'new.recap',
 'misc/notes': 'misc',
 'New/Recap': 'new.recap',
 'TL': 'tl',
 'Toe 8': None,
 'paint mark': 'paint.mark',
 'meters': 'met

We'll use the dict, *uniquecols2dict* to rename the synonymous columns in our file....once we read them in,
that is.

<a id='ReadingAppendingData'></a>

# Reading and Appending Data
[Top](#TOC)

Now we use the function *readnclean* to actually read in the source files, drop unnecessary columns and renaming columns according to a dictionary.

Here is an example of how *readnclean* works.  The code can be found in [Functions](#functions).

In [37]:
readnclean(rawfiles_ss[0],uniquecols2dict,dtype=str).head()

,species,toes,date,sex,svl,tl,rtl,mass,paint.mark,location,meters,new.recap,painted,misc,vial
0,sv,2,2012-06-02 00:00:00,m,53,72,0,4.4,w19b,pine at top of site,nan,???,yes,actually caught on way down; toe #2 may be nat...,nan
1,sv,nan,2012-05-29 00:00:00,nan,nan,nan,nan,nan,???,1falls,nan,didn't catch,nan,nan,nan
2,sc,11,2012-06-01 00:00:00,F,98,129,0,35,.t,bottom site Rt side,nan,new,yes,BRIA CAUGHT IT!!!!!! :),12-41
3,cn ex,1-7,2012-05-27 00:00:00,f,89,165,75,19,w.a,sb at CCC,nan,new,yes,nan,12-27
4,sj,5-11-16,2012-05-24 00:00:00,F,62,85,0,8.4,w1c,5m^bottom site,nan,new,yes,nan,12-01


We'll create a df, *df*, with no data, but columns from our desired columns, *i.e.* the keys for coldict, as a placeholder to which we can append new data.

In [38]:
df = pd.DataFrame(columns=coldict.keys())
df

,species,date,sex,svl,tl,rtl,mass,paint.mark,location,meters,new.recap,painted,misc,vial,autotomized,sighting,toes


Now we will read in all of the raw_ss files clean the column names and concatenate them into one large df.

In [39]:
for file in rawfiles_ss:
    df = pd.concat([df,readnclean(file,uniquecols2dict)],sort=True)
    print(df.shape[0])
print("\n\nFinal df has {} columns and {} rows.".format(df.shape[1],df.shape[0]))
df.head()

85
883
986
1188
1351
1485
1582
1646
1808
1907
2013
2492
2733


Final df has 17 columns and 2733 rows.


,autotomized,date,location,mass,meters,misc,new.recap,paint.mark,painted,rtl,sex,sighting,species,svl,tl,toes,vial
0.0,NaN,2012-06-02 00:00:00,pine at top of site,4.4,NaN,actually caught on way down; toe #2 may be nat...,???,w19b,yes,0,m,NaN,sv,53,72,2,NaN
1.0,NaN,2012-05-29 00:00:00,1falls,NaN,NaN,NaN,didn't catch,???,NaN,NaN,NaN,NaN,sv,NaN,NaN,NaN,NaN
2.0,NaN,2012-06-01 00:00:00,bottom site Rt side,35,NaN,BRIA CAUGHT IT!!!!!! :),new,.t,yes,0,F,NaN,sc,98,129,11,12-41
3.0,NaN,2012-05-27 00:00:00,sb at CCC,19,NaN,NaN,new,w.a,yes,75,f,NaN,cn ex,89,165,1-7,12-27
4.0,NaN,2012-05-24 00:00:00,5m^bottom site,8.4,NaN,NaN,new,w1c,yes,0,F,NaN,sj,62,85,5-11-16,12-01


For the *rawdata_ms* files we will apply a similar logic

<a id='ExportingData'></a>

# Exporting Data
[Top](#TOC)

Here we call the function, *namefile*, to create a timestamped name for file to be exported.  You can find the code for *namefile* in [Functions](#functions).

In [40]:
filename = namefile('Appended and Trimmed CC Data 2000-2017')
os.chdir(outputBig)
df.to_csv(filename,index = False)
print("\'{}\' has been saved to \'{}\' and the corresponding drive google drive location."\
      .format(filename, outputBig))

'Appended and Trimmed CC Data 2000-2017_2018-12-22 22hrs33min.csv' has been saved to 'S:/Chris/TailDemography/TailDemography/Cleaned Combined Data' and the corresponding drive google drive location.
